In [3]:
moviesPath = "Spark/Exercises/Ex_45/movies.txt"
preferencesPath = "Spark/Exercises/Ex_45/preferences.txt"
watchedMoviesPath = "Spark/Exercises/Ex_45/watchedmovies.txt"
moviesRDD = sc.textFile(moviesPath)
# MID, title, genre
preferencesRDD = sc.textFile(preferencesPath)
# UID, genre
watchedMoviesRDD = sc.textFile(watchedMoviesPath)
# UID, MID, startTS, endTS

In [53]:
# (MID, UID)
MID_UID_RDD = watchedMoviesRDD.map(lambda line : (line.split(",")[1],line.split(",")[0]))
# (MID, genre)
MID_genre_RDD = moviesRDD.map(lambda line : (line.split(",")[0],line.split(",")[2]))
# (MID, (UID, genre)) -> (UID, watchedGenre) with repetitions of UID
UID_WG_RDD = MID_UID_RDD.join(MID_genre_RDD).values()
# (UID, likedGenre) with repetitions of UID
UID_likedGenre_RDD = preferencesRDD.map(lambda line : (line.split(",")[0],line.split(",")[1]))
# (UID, ([likedGenres], [watchedGenres]))
UID_listLGListWG_RDD = UID_likedGenre_RDD.cogroup(UID_WG_RDD).mapValues(lambda t : (list(t[0]), list(t[1])))
UID_listLGListWG_RDD.collect()

[('user2',
  (['Action'], ['Comedy', 'Comedy', 'Comedy', 'Comedy', 'Action', 'Comedy'])),
 ('user1',
  (['Animation', 'Comedy'], ['Comedy', 'Animation', 'Comedy', 'Comedy']))]

In [58]:
def filtUser(x):
    likedGenres = x[1][0]
    watchedGenres = x[1][1]
    nonMatch = 0
    for currGenre in watchedGenres:
        if currGenre not in likedGenres:
            nonMatch += 1
    if 100*nonMatch/len(watchedGenres) > 60:
        return True
    else:
        return False

# (mislUID, ([likedGenres], [watchedGenres]))
mislUID_listLGListWG_RDD = UID_listLGListWG_RDD.filter(filtUser)

In [65]:
def mapFunc2(x):
    likedGenres = x[1][0]
    watchedGenres = x[1][1]
    localDict = {}
    finalGenresList = []
    
    for currWatchedGenre in watchedGenres:
        if currWatchedGenre not in likedGenres:
            if currWatchedGenre not in localDict:
                localDict[currWatchedGenre] = 1
            else:
                localDict[currWatchedGenre] += 1
    
    for genre in localDict.keys():
        if localDict[genre] >= 5:
            finalGenresList.append(genre)
    
    return (x[0], finalGenresList)
    
    
finalRDD = mislUID_listLGListWG_RDD.map(mapFunc2)
finalRDD.collect()

[('user2', ['Comedy'])]